# import libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import numpy as np
from collections import Counter
from keras.models import load_model
from keras.datasets import cifar10

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
drive_path = '/content/drive/MyDrive/UTS_MKTK'

Mounted at /content/drive/


# read dataset: CIFAR-10

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

170508288/170498071 [==============================] - 2s 0us/step


In [4]:
for i in [0, 1, 8, 9]:
  idx = (train_labels == i).reshape(train_images.shape[0])
  train_img = train_images[idx][:500]
  val_img = train_images[idx][500:750]
  test_img = train_images[idx][750:800]
  if i == 0:
    airplane_img_tr = train_img
    airplane_img_val = val_img
    airplane_img_te = test_img    
  elif i == 1:
    automobile_img_tr = train_img
    automobile_img_val = val_img
    automobile_img_te = test_img
  elif i == 8:
    ship_img_tr = train_img
    ship_img_val = val_img
    ship_img_te = test_img
  else:
    truck_img_tr = train_img
    truck_img_val = val_img
    truck_img_te = test_img

training_image = np.concatenate((airplane_img_tr, automobile_img_tr, ship_img_tr, truck_img_tr), axis=0)
validation_image = np.concatenate((airplane_img_val, automobile_img_val, ship_img_val, truck_img_val), axis=0)
testing_image = np.concatenate((airplane_img_te, automobile_img_te, ship_img_te, truck_img_te), axis=0)

In [5]:
A = []
B = []
C = []
D = []
for i in range(500):
  A.append([0])
  B.append([1])
  C.append([2])
  D.append([3])

A = np.array(A)
B = np.array(B)
C = np.array(C)
D = np.array(D)

training_label = np.concatenate((A, B, C, D), axis=0)

In [6]:
A = []
B = []
C = []
D = []
for i in range(250):
  A.append([0])
  B.append([1])
  C.append([2])
  D.append([3])

A = np.array(A)
B = np.array(B)
C = np.array(C)
D = np.array(D)

validation_label = np.concatenate((A, B, C, D), axis=0)

In [7]:
A = []
B = []
C = []
D = []
for i in range(50):
  A.append([0])
  B.append([1])
  C.append([2])
  D.append([3])

A = np.array(A)
B = np.array(B)
C = np.array(C)
D = np.array(D)

testing_label = np.concatenate((A, B, C, D), axis=0)

In [8]:
print(training_image.shape)
print(validation_image.shape)
print(testing_image.shape)

print(training_label.shape)
print(validation_label.shape)
print(testing_label.shape)

class_names = ['airplane', 'automobile', 'ship', 'truck']

(2000, 32, 32, 3)
(1000, 32, 32, 3)
(200, 32, 32, 3)
(2000, 1)
(1000, 1)
(200, 1)


In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((training_image, training_label))
test_ds = tf.data.Dataset.from_tensor_slices((testing_image, testing_label))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_image, validation_label))

# define Preprocessing

In [10]:
def process_images(image, label):
    image = tf.image.per_image_standardization(image) #strandarisasi
    image = tf.image.resize(image, (224,224))         #resize
    return image, label

In [11]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 2000
Test data size: 200
Validation data size: 1000


In [12]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=10, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=10, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=10, drop_remainder=True))

# model

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense, Activation, Add, Flatten
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
from tensorflow.keras import activations
from tensorflow.keras.regularizers import L2
import tensorflow.keras.backend as K

In [14]:
def res_identity(x, filters): 
  x_skip = x 
  f1, f2 = filters

  #first block 
  x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=L2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=L2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # third block activation used after adding the input
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=L2(0.001))(x)
  x = BatchNormalization()(x)
  # x = Activation(activations.relu)(x)

  # add the input 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=L2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=L2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=L2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=L2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

def resnet50():

  input_im = Input(shape=(224, 224, 3)) # cifar 10 images size
  x = ZeroPadding2D(padding=(3, 3))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPool2D((3, 3), strides=(2, 2))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 3rd stage

  x = res_conv(x, s=2, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))

  # 4th stage

  x = res_conv(x, s=2, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))

  # 5th stage

  x = res_conv(x, s=2, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))

  # ends with average pooling and dense connection

  x = AvgPool2D((2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(4, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model

In [15]:
model = resnet50()

In [18]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()

#Defining callbacks to get the best model possible
my_calls = [keras.callbacks.EarlyStopping(monitor='val_loss',patience=30),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, min_lr=1.5e-5),
            keras.callbacks.ModelCheckpoint(drive_path + "/Model3.h5",verbose=1,save_best_only=True),
            keras.callbacks.TensorBoard(run_logdir)]

In [19]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.Adam(lr=0.001), metrics=['accuracy'])
model.summary()

Model: "Resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
hist = model.fit(train_ds, epochs=30, validation_data=validation_ds,
                 validation_freq=1, callbacks=my_calls)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/30
200/200 [==============================] - ETA: 0s - loss: 23.4452 - accuracy: 0.3185
Epoch 00001: val_loss improved from inf to 20.29151, saving model to /content/drive/MyDrive/UTS_MKTK/Model3.h5
200/200 [==============================] - 108s 342ms/step - loss: 23.4452 - accuracy: 0.3185 - val_loss: 20.2915 - val_accuracy: 0.3860 - lr: 0.0010
Epoch 2/30
200/200 [==============================] - ETA: 0s - loss: 16.1408 - accuracy: 0.4175
Epoch 00002: val_loss improved from 20.29151 to 16.33340, saving model to /content/drive/MyDrive/UTS_MKTK/Model3.h5
200/200 [==============================] - 70s 345ms/step - loss: 16.1408 - accuracy: 0.4175 - val_loss: 16.3334 - val_accuracy: 0.4470 - lr: 0.0010
Epoch 3/30
200/200 [==============================] - ETA: 0s - loss: 12.4832 - accuracy: 0.4830
Epoch 00003: val_loss improved from 16.33340 to 10.78057, saving model to /content/drive/MyDrive/UTS_MKTK/Model3.h5
200/200 [==============================] - 70s 347ms/step - loss: 1

KeyboardInterrupt: ignored

# load model

In [21]:
model = load_model(drive_path + "/Model3.h5")

In [22]:
model.evaluate(test_ds)

20/20 [==============================] - 3s 99ms/step - loss: 2.7427 - accuracy: 0.5550


[2.7426645755767822, 0.5550000071525574]